In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
print("g")

g


In [1]:
!git clone https://github.com/deepseek-ai/Janus.git

Cloning into 'Janus'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 115 (delta 23), reused 22 (delta 14), pack-reused 74 (from 1)
Receiving objects: 100% (115/115), 7.16 MiB | 46.40 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [2]:
%cd Janus

/kaggle/working/Janus


In [3]:
!pip install -e .

Obtaining file:///kaggle/working/Janus
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for janus (pyproject.toml) ... done
  Created wheel for janus: filename=janus-1.0.0-0.editable-py3-none-any.whl size=15932 sha256=34f886cd6d70ec614d60fb68c6a53a7ab60d71a38efd7479af07baf732e7e903
  Stored in directory: /tmp/pip-ephem-wheel-cache-a_wkzwq4/wheels/cb/20/db/a2e58bf4a9076003af4cdd6a476a8ae1a9936d9bf84bbcff86
Successfully built janus


In [3]:
from flash_attn import flash_attn_qkvpacked_func, flash_attn_func

ModuleNotFoundError: No module named 'flash_attn'

In [4]:
!pip install flash-attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 41.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.7.3-cp310-cp310-linux_x86_64.whl size=191342964 sha256=cd0ead6ad92bff21f90cf0aecdf859bc7145bf4e7dc8ed1fdc3cc38299651ce4
  Stored in directory: /root/.cache/pip/wheels/85/d7/10/a74c9fe5ffe6ff306b27a220b2bf2f37d907b68fdcd138cdda
Successfully built flash-attn


In [9]:
from flash_attn import flash_attn_qkvpacked_func, flash_attn_func
flash_attn_qkvpacked_func(qkv, dropout_p=0.0, softmax_scale=None, causal=False,
window_size=(-1, -1), alibi_slopes=None, deterministic=False):


SyntaxError: invalid syntax (<ipython-input-9-1c0b26d456ed>, line 3)

In [13]:

import torch
from transformers import AutoModelForCausalLM
from janus.models import MultiModalityCausalLM, VLChatProcessor
from janus.utils.io import load_pil_images

# specify the path to the model
model_path = "deepseek-ai/Janus-Pro-1B"
vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(
    model_path, trust_remote_code=True
)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

conversation = [
    {
        "role": "What is in the image?",
        "content": f"<image_placeholder>\n{question}",
        "images": /kaggle/working/Janus/images/equation.png,
    },
    {"role": "<|Assistant|>", "content": ""},
]

# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation, images=pil_images, force_batchify=True
).to(vl_gpt.device)

# # run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# # run the model to get the response
outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True,
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
print(f"{prepare_inputs['sft_format'][0]}", answer)


SyntaxError: invalid syntax (<ipython-input-13-ccfd92ba1400>, line 20)

In [12]:
import torch
from transformers import AutoModelForCausalLM
from janus.models import MultiModalityCausalLM, VLChatProcessor
from janus.utils.io import load_pil_images
from PIL import Image
import requests
from io import BytesIO


# specify the path to the model
model_path = "deepseek-ai/Janus-Pro-1B"
vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(
    model_path, trust_remote_code=True
)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()


# --- Define question and image here ---
question = "What is in the image?"  # Define your question here
# Example: Load image from a URL
image_url = "https://github.com/deepseek-ai/Janus/blob/main/images/equation.png"  # Replace with your image url
response = requests.get(image_url)
image = Image.open(BytesIO(response.content)).convert("RGB")

# --- Now use question and image ---
conversation = [
    {
        "role": "<|User|>",
        "content": f"<image_placeholder>\n{question}",
        "images": [image],
    },
    {"role": "<|Assistant|>", "content": ""},
]

# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation, images=pil_images, force_batchify=True
).to(vl_gpt.device)

# # run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# # run the model to get the response
outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True,
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
print(f"{prepare_inputs['sft_format'][0]}", answer)

Some kwargs in processor config are unused and will not have any effect: sft_format, mask_prompt, num_image_tokens, add_special_token, image_tag, ignore_id. 


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7ed70466cb80>

In [14]:
import torch
from transformers import AutoModelForCausalLM
from janus.models import MultiModalityCausalLM, VLChatProcessor
from janus.utils.io import load_pil_images

# specify the path to the model
model_path = "deepseek-ai/Janus-Pro-1B" 
vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(
    model_path, trust_remote_code=True
)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

# تحديد مسار الصورة والسؤال
image_path = "/kaggle/working/Janus/images/equation.png"
question = "What is in the image?"

# بناء قائمة المحادثة
conversation = [
    {
        "role": "<|User|>",  # تم تغيير دور المستخدم إلى "<|User|>"
        "content": f"<image_placeholder>\n{question}",
        "images": [image_path], # تم وضع مسار الصورة في قائمة
    },
    {"role": "<|Assistant|>", "content": ""},
]

# تحميل الصور وإعداد المدخلات
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation, images=pil_images, force_batchify=True
).to(vl_gpt.device)

# تشغيل مشفر الصور للحصول على الـ embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# تشغيل النموذج للحصول على الاستجابة
outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True,
)

# فك تشفير الاستجابة
answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
print(f"{prepare_inputs['sft_format'][0]}", answer)

Some kwargs in processor config are unused and will not have any effect: sft_format, mask_prompt, num_image_tokens, add_special_token, image_tag, ignore_id. 


You are a helpful language and vision assistant. You are able to understand the visual content that the user provides, and assist the user with a variety of tasks using natural language.

<|User|>: <image_placeholder>
What is in the image?

<|Assistant|>: The image shows a mathematical expression involving a summation and a product. The expression is:

\[
A_n = \left| a_0 \right| \left| 1 + \frac{3}{4} \sum_{k=1}^{n} \left( \frac{4}{9} \right)^k \right|
\]

This is a representation of the absolute value of the sum of the first \( n \) positive terms of the sequence \( a_n = \left| a_0 \right| \left| 1 + \frac{3}{4} \right| \left| \left( \frac{4}{9} \right)^n \right| \).
